# OpenAI Swarm: A Framework for Multi-Agent Orchestration 

In [ ]:
!pip install git+https://github.com/openai/swarm.git

## Main Agent using Tools

In [ ]:
from swarm import Agent, Swarm
from typing import List, Callable

AGENT_SYSTEM_PROMPT = """
# Task
Tu nombre es Gabriela y sos un asistente de IA diseñado para ayudar al equipo de Pampa Labs. 
                      
# Guidelines                   
Tus respuestas deben ser:

1. Amigables y accesibles, usando un tono cálido
2. Concisas y al grano, evitando verbosidad innecesaria
3. Útiles e informativas, proporcionando información precisa
4. Respetuosas de la privacidad del usuario y los límites éticos

Solo puedes ayudar usando las herramientas disponibles y con pedidos que vengan de miembros del equipo. Todo lo que no se pueda responder usando las herramientas, debes decir que no puedes ayudar y disculparte.
"""

# I had to convert Pydantic models to function arguments with corresponding docstrings
# This involves extracting fields from the Pydantic models and creating function
# parameters with appropriate type hints and descriptions in the docstring.
def set_meal_tool(context_variables, meal: str, date: str):
    """
    Sets the meal plan for a specific date.

    Args:
        meal (str): The name of the meal.
        date (str): The date of the meal plan.
    """
    return f"Meal plan created and sent to the provider: {meal} for {date} by team member {context_variables['id']}"

def get_expenses_tool(context_variables):
    """
    Retrieves all pending expenses
    """

    # Mock implementation for testing purposes
    mock_expenses = {
        "user1": {
            'expenses': [
                {
                "expense_type": "food",
                "date": "2023-05-01",
                "total_value": 50.0,
                "state": "pending"
            },
            {
                "expense_type": "coffee",
                "date": "2023-05-02",
                "total_value": 5.0,
                "state": "pending"
                }
            ]
        }
    }
    # Functions called by an Agent should return a type that has __str__
    return f"Expenses retrieved: {mock_expenses[context_variables['id']]['expenses']}"

class GabyAgent:
    def __init__(self,
        prompt = AGENT_SYSTEM_PROMPT,
        model_name: str = "gpt-4o",
        tools: List[Callable] = [],
    ):

        self.agent = Agent(
            name="Main Agent",
            instructions=prompt,
            functions=tools,
            model=model_name
        )
        self.client = Swarm()

    def invoke(self, id, messages, debug=False):
        response = self.client.run(
            agent=self.agent,
            messages=messages,
            debug=debug,
            context_variables={"id": id}
        )
        return response.messages[-1]["content"]
    
    def stream(self, id, messages, debug=False):
        stream = self.client.run(
            agent=self.agent,
            messages=messages,
            debug=debug,
            context_variables={"id": id},
            stream=True
        )
        for chunk in stream:
            yield chunk

In [ ]:
gaby_agent = GabyAgent(tools=[set_meal_tool, get_expenses_tool])
response_content = gaby_agent.invoke(
    id="user1",
    messages=[
        {"role": "user", "content": "I want to set the meal plan for the date 2024-05-01. The meal is lasagna."},
        {"role": "user", "content": "I want to get the expenses"}
    ],
    debug=False
)
print(response_content)

### Streaming response

In [ ]:
for chunk in gaby_agent.stream(
    id="user1",
    messages=[
        {"role": "user", "content": "I want to set the meal plan for the date 2024-05-01. The meal is lasagna."},
        {"role": "user", "content": "I want to get the expenses"}
    ],
    debug=True
):
    print(chunk)

## Handoff Functions

In [ ]:
meal_agent: Agent = Agent(
    name="Meal Agent",
    instructions="""
    Eres un experto en planificar pedidos de comidas para el Equipo, tus respuesta deben ser claras, directas y en argentino.
    """,
    functions=[set_meal_tool],
    model="gpt-4o"
)

def transfer_to_meal_agent() -> Agent:
    """
    Transfer control to the Meal Agent for handling meal-related tasks.
    This function is used when a user request is specifically about meal planning or orders.
    """
    return meal_agent

expenses_agent: Agent = Agent(
    name="Expenses Agent",  
    instructions="""
    Eres un experto en gestionar los gastos del Equipo. Tus respuestas deben ser en argentino.
    """,
    functions=[get_expenses_tool],
    model="gpt-4o"
)

def transfer_to_expenses_agent() -> Agent:
    """
    Transfer control to the Expenses Agent for handling expenses-related tasks.
    """
    return expenses_agent

In [ ]:
gaby_agent = GabyAgent(tools=[transfer_to_meal_agent, transfer_to_expenses_agent])
response_content = gaby_agent.invoke(
    id="user1",
    messages=[
        {"role": "user", "content": "I want to set the meal plan for the date 2024-05-01. The meal is lasagna."},
    ],
    debug=True
)
print(response_content)

response_content = gaby_agent.invoke(
    id="user1",
    messages=[
        {"role": "user", "content": "I want to get the expenses"}
    ],
    debug=True
)
print(response_content)